# SQL Windows Functions
L'objectif de ce notebook est de vous introduire aux notions de windows function. Celui-ci n'a donc pas pour but d'être exhaustif mais de vous donner des clés de compréhensions pour ce sujet. 


## Qu'est ce qu'une window function ? 
Une window function est fonction SQL ou le résultat est tiré d'un d'une "fenêtre" (un set d'une ou plusieurs lignes). En gros, ces fonctions nous donnent des outils pour filter et/ou ordonner un sous ensemble de données sur lesquels on peut appliquer des fonctions.

En plus de ces outils de "fenêtrage", nous avons aussi un ensemble de fonction bien spécifique (numéroter des lignes, sélectionner la première valeur d'un sous ensemble ordonner, sélectionner la valeur suivant d'un sous ensemble ordonner...) 

Nous verrons ici des uses cases et des exercices pour vous faire la main.

In [1]:
# configuration de l'environnement 
from google.cloud import bigquery
import os 
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = \
    "/Users/garantme/Downloads/datawarehouse-267911-7a35fd65f9dd.json"

%load_ext google.cloud.bigquery

### Premier use case : les dates de premier paiement
Commençons doucement, nous souhaitons récupérer la date du premier paiement associé à une application

In [2]:
%%bigquery

# Requête pour récupérer tous les paiements de chaque application
SELECT 
    AI.applicationid, 
    I.paymentsucceededat
FROM `datawarehouse-267911.db_production_console.garantme_ApplicationInvoice` AI
INNER JOIN `datawarehouse-267911.db_production_console.garantme_PaymentPlan` PP ON AI.id=PP.applicationinvoiceid 
INNER JOIN `datawarehouse-267911.db_production_console.garantme_Installment` I ON PP.id=I.paymentplanid
INNER JOIN `datawarehouse-267911.db_production_console.garantme_InstallmentPaymentStatus` IPS ON I.installmentpaymentstatusid=IPS.id
WHERE 
    AI.deletedat IS NULL
    AND PP.deletedat IS NULL 
    AND I.deletedat IS NULL
    AND IPS.name IN('Succeeded')
LIMIT 10

Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 11.60rows/s]


,applicationid,paymentsucceededat
0,231124,2021-07-12 18:49:53.956000+00:00
1,219658,2021-07-12 18:51:34.824000+00:00
2,226543,2021-07-12 18:11:49.217000+00:00
3,225371,2021-07-12 18:10:44.814000+00:00
4,233241,2021-07-12 18:02:07.007000+00:00
5,228541,2021-07-12 17:16:59.498000+00:00
6,215127,2021-07-12 17:16:45.697000+00:00
7,234094,2021-07-12 18:12:34.122000+00:00
8,229997,2021-07-12 19:04:41.227000+00:00
9,228647,2021-07-12 19:03:30.771000+00:00


In [3]:
%%bigquery

# Requête pour récupérer les premiers paiement
SELECT 
    AI.applicationid, 
    FIRST_VALUE(I.paymentsucceededat) OVER (
        PARTITION BY AI.applicationid
        ORDER BY I.paymentsucceededat
    ) AS premier_paiement
FROM `datawarehouse-267911.db_production_console.garantme_ApplicationInvoice` AI
INNER JOIN `datawarehouse-267911.db_production_console.garantme_PaymentPlan` PP ON AI.id=PP.applicationinvoiceid 
INNER JOIN `datawarehouse-267911.db_production_console.garantme_Installment` I ON PP.id=I.paymentplanid
INNER JOIN `datawarehouse-267911.db_production_console.garantme_InstallmentPaymentStatus` IPS ON I.installmentpaymentstatusid=IPS.id
WHERE 
    AI.deletedat IS NULL
    AND PP.deletedat IS NULL 
    AND I.deletedat IS NULL
    AND IPS.name IN('Succeeded')
LIMIT 10

Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 12.03rows/s]


,applicationid,premier_paiement
0,111,2018-02-23 23:04:12+00:00
1,377,2018-03-14 07:34:09+00:00
2,500,2018-04-26 15:31:04+00:00
3,500,2018-04-26 15:31:04+00:00
4,546,2021-03-30 09:31:19.319000+00:00
5,616,2018-04-15 21:01:07+00:00
6,681,2018-05-25 13:57:10+00:00
7,757,2018-05-14 16:36:20+00:00
8,757,2018-05-14 16:36:20+00:00
9,757,2018-05-14 16:36:20+00:00


Regardons ce que nous venons de faire.
La fonction "OVER" permet de créer notre fameuse fenêtre (le sous ensemble sur lequel nous voulons obtenir un résultat). Ce OVER possède trois arguments : 
 - PARTITION BY
 - ORDER BY
 - ROWS/RANGE BETWEEN
 
Mettons le rows between de côté pour le moment. Dans l'exemple ci-dessus, nous avons spécifié PARTITION BY (qui revient à une clause GROUP BY) et un ORDER BY. On créer donc une partition par application. Cette partition est trié par date de paiement. Pour cette partition triée, on sélectionne la première valeur avec "FIRST_VALUE"

Pour le plus sagaces d'entre vous, vous vous dites qu'on aurait pu faire ça avec un GROUP BY et un MIN. Oui. Les choses se compliqueront par la suite. Notons toutefois une première différence avec un GROUP BY, la windows function renvoie une résultat par input, alors que le GROUP BY aggrège.

Compliquons tout de suite les choses pour montrer la vraie plus value de ce genre de méthode : nous souhaitons numéroter les paiements en fonction de leur date de survenance

In [4]:
%%bigquery

# Requête pour récupérer les premiers paiement
SELECT 
    AI.applicationid, 
    ROW_NUMBER() OVER (
        PARTITION BY AI.applicationid
        ORDER BY I.paymentsucceededat
    ) AS numero_paiement
FROM `datawarehouse-267911.db_production_console.garantme_ApplicationInvoice` AI
INNER JOIN `datawarehouse-267911.db_production_console.garantme_PaymentPlan` PP ON AI.id=PP.applicationinvoiceid 
INNER JOIN `datawarehouse-267911.db_production_console.garantme_Installment` I ON PP.id=I.paymentplanid
INNER JOIN `datawarehouse-267911.db_production_console.garantme_InstallmentPaymentStatus` IPS ON I.installmentpaymentstatusid=IPS.id
WHERE 
    AI.deletedat IS NULL
    AND PP.deletedat IS NULL 
    AND I.deletedat IS NULL
    AND IPS.name IN('Succeeded')
LIMIT 10

Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 11.29rows/s]


,applicationid,numero_paiement
0,318,1
1,1142,1
2,1911,1
3,2295,1
4,4634,1
5,6033,1
6,6352,1
7,6572,1
8,10147,1
9,10777,1


Avant de passer à un autre use case, faisont un arrêt sur les clause ROWS BETWEEN ou RANGE BETWEEN. Celles-ci nous permette de mettre des bornes à nos fenêtres. Les bornes peuvent être indiquées selon une valeur (RANGE BETWEEN) ou selon un nombre de ligne (ROWS BETWEEN).

Nouvel exercice, nous souhaitons récupérer la somme payée pour une application sur les 2 dernieres années glissantes

In [47]:
%%bigquery

# Juste pour visualiser une application avec plusieurs paiement
SELECT 
    AI.applicationid,
    AI.amountwithvat,
    I.paymentsucceededat
FROM `datawarehouse-267911.db_production_console.garantme_ApplicationInvoice` AI
INNER JOIN `datawarehouse-267911.db_production_console.garantme_PaymentPlan` PP ON AI.id=PP.applicationinvoiceid 
INNER JOIN `datawarehouse-267911.db_production_console.garantme_Installment` I ON PP.id=I.paymentplanid
WHERE 
    AI.applicationid = 403

Downloading: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00,  4.11rows/s]


,applicationid,amountwithvat,paymentsucceededat
0,403,93940,NaT
1,403,93940,2018-03-23 10:42:22+00:00
2,403,93940,2019-02-26 00:00:00+00:00
3,403,93940,2020-02-26 00:00:00+00:00


In [48]:
%%bigquery

# Requête pour récupérer la somme de paiements sur 365 jour glissant
SELECT 
    AI.applicationid, 
    I.paymentsucceededat,
    SUM(AI.amountwithvat) OVER (
        PARTITION BY AI.applicationid
        ORDER BY UNIX_DATE(DATE(I.paymentsucceededat))
        RANGE BETWEEN 730 PRECEDING AND CURRENT ROW
    ) AS somme_payee
FROM `datawarehouse-267911.db_production_console.garantme_ApplicationInvoice` AI
INNER JOIN `datawarehouse-267911.db_production_console.garantme_PaymentPlan` PP ON AI.id=PP.applicationinvoiceid 
INNER JOIN `datawarehouse-267911.db_production_console.garantme_Installment` I ON PP.id=I.paymentplanid
INNER JOIN `datawarehouse-267911.db_production_console.garantme_InstallmentPaymentStatus` IPS ON I.installmentpaymentstatusid=IPS.id
WHERE 
    AI.deletedat IS NULL
    AND PP.deletedat IS NULL 
    AND I.deletedat IS NULL
    AND IPS.name IN('Succeeded')
    AND AI.applicationid = 403
LIMIT 10

Downloading: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  3.11rows/s]


,applicationid,paymentsucceededat,somme_payee
0,403,2018-03-23 10:42:22+00:00,93940
1,403,2019-02-26 00:00:00+00:00,187880
2,403,2020-02-26 00:00:00+00:00,281820


### Deuxième use case
Lag and Lead

### Troisième use case 
